In [1]:
import inspect
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_symmetry import benz_Symm

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis='sto-3g',symmetry = 'D6v') #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})

In [3]:
BenzHF=scf.RHF(Benz)

In [4]:
%time BenzHF.scf()

converged SCF energy = -227.890711338274
CPU times: user 1.25 s, sys: 39.1 ms, total: 1.29 s
Wall time: 466 ms


-227.8907113382736

In [5]:
%time d1,d2,d3= alch_deriv(BenzHF,[[0],[1]])
d1,d2,d3

CPU times: user 279 ms, sys: 23.4 ms, total: 302 ms
Wall time: 97.8 ms


(-14.55480844327413, -1.079627074618507, 0.2602164525966672)

In [6]:
dl=1e-3
fdhf1=scf.RHF(FcM(fcs=[[0,3],[dl,dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e1=fdhf1.scf()
fdhf2=scf.RHF(FcM(fcs=[[0,3],[-dl,-dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e2=fdhf2.scf()
e0=BenzHF.scf()

converged SCF energy = -227.890711338273


In [7]:
fdhf3=scf.RHF(FcM(fcs=[[0,3],[2*dl,2*dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e3=fdhf3.scf()
fdhf4=scf.RHF(FcM(fcs=[[0,3],[-2*dl,-2*dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e4=fdhf4.scf()

In [8]:
(e1-e2)/2/dl,(e1+e2-2*e0)/dl**2,(e3/2-e1+e2-e4/2)/dl**3

(-15.697750917723852, -1.2082362559340254, 0.18114576505467994)

In [9]:
%time d1,d2,d3= alch_deriv(BenzHF,[[0,3],[1,1]])
d1,d2,d3

CPU times: user 319 ms, sys: 43.7 ms, total: 362 ms
Wall time: 122 ms


(-15.69775094105993, -1.2082383098489955, 0.180764323767294)

In [10]:
%time af1= aaff(BenzHF,[1,0,0,0,0,0,0,0,0,0,0,0,0])

CPU times: user 11.2 s, sys: 131 ms, total: 11.3 s
Wall time: 8.57 s


In [11]:
Benz.symmetry

'D6v'

In [12]:
from AP_symmetry import rotate_grad,rotate_matrix

In [13]:
g=BenzHF.Gradients().grad_nuc()
g

array([[ 2.96057623e-12, -8.58254608e+00,  7.79752832e-08],
       [ 2.76043077e-13, -2.54741647e+00, -1.14081458e-09],
       [-7.43270294e+00, -4.29127305e+00,  1.21872018e-07],
       [-2.20612738e+00, -1.27370824e+00, -7.74666156e-09],
       [-7.43270294e+00,  4.29127305e+00, -9.64438761e-08],
       [-2.20612738e+00,  1.27370824e+00, -5.62362280e-08],
       [-2.33539577e-12,  8.58254608e+00,  6.95524577e-08],
       [-1.72069303e-12,  2.54741647e+00,  2.13607535e-08],
       [ 7.43270294e+00,  4.29127305e+00,  4.55789859e-08],
       [ 2.20612738e+00,  1.27370824e+00, -8.18970519e-09],
       [ 7.43270294e+00, -4.29127305e+00, -1.42698422e-07],
       [ 2.20612738e+00, -1.27370824e+00, -2.38837909e-08]])

In [14]:
rotate_grad(g,2)

array([[ 7.43270294e+00, -4.29127305e+00, -1.42698422e-07],
       [ 2.20612738e+00, -1.27370824e+00, -2.38837909e-08],
       [ 2.96057623e-12, -8.58254608e+00,  7.79752832e-08],
       [ 2.76043077e-13, -2.54741647e+00, -1.14081458e-09],
       [-7.43270294e+00, -4.29127305e+00,  1.21872018e-07],
       [-2.20612738e+00, -1.27370824e+00, -7.74666156e-09],
       [-7.43270294e+00,  4.29127305e+00, -9.64438761e-08],
       [-2.20612738e+00,  1.27370824e+00, -5.62362280e-08],
       [-2.33539577e-12,  8.58254608e+00,  6.95524577e-08],
       [-1.72069303e-12,  2.54741647e+00,  2.13607535e-08],
       [ 7.43270294e+00,  4.29127305e+00,  4.55789859e-08],
       [ 2.20612738e+00,  1.27370824e+00, -8.18970519e-09]])

In [15]:
from scipy.spatial.transform import Rotation as R

In [16]:
rv=-np.asarray([0,0,1])*np.pi*2/3
r = R.from_rotvec(rv)

In [17]:
np.allclose(rotate_grad(r.apply(g),4),g,atol=1e-6)

True

In [18]:
np.set_printoptions(precision=3,suppress=True)

## Symmetry operations for Benzene $D_{6h}$ axis 
I need to <br>
1) dP (i) -> rotate dP (0/1) <br>
2) af(i) = rotate af (0/1)<br>
3) dV(i) =rotate dV(0/1)<br>

In [19]:
import itertools
%autoreload 2
from AP_class import DeltaV,parse_charge,alchemy_cphf_deriv,make_dP
from AP_class import APDFT_perturbator as AP

In [20]:
bs=benz_Symm(BenzHF)

In [21]:
ap=AP(BenzHF,sites=[1,2,0],symm=bs)

In [22]:
ap.af(1),ap.af(9)

No AF found for atom 1. Calculating it from code


(array([[-0.   ,  0.206,  0.   ],
        [ 0.   , -0.115, -0.   ],
        [-0.021, -0.035, -0.   ],
        [-0.001, -0.004, -0.   ],
        [ 0.005, -0.   , -0.   ],
        [-0.001, -0.005, -0.   ],
        [-0.   , -0.006,  0.   ],
        [-0.   ,  0.003, -0.   ],
        [-0.005, -0.   , -0.   ],
        [ 0.001, -0.005, -0.   ],
        [ 0.021, -0.035, -0.   ],
        [ 0.001, -0.004, -0.   ]]), array([[-0.003,  0.005, -0.   ],
        [ 0.005,  0.001, -0.   ],
        [ 0.005,  0.003,  0.   ],
        [-0.003, -0.002, -0.   ],
        [ 0.003, -0.005, -0.   ],
        [ 0.004,  0.003, -0.   ],
        [ 0.02 ,  0.035, -0.   ],
        [ 0.003,  0.003, -0.   ],
        [-0.179, -0.103,  0.   ],
        [ 0.1  ,  0.058, -0.   ],
        [ 0.04 , -0.   , -0.   ],
        [ 0.004,  0.001, -0.   ]]))

In [28]:
ap.sites=[0,2,4,6,8,10]

In [29]:
ap.perturb()

In [30]:
ap.dV(2)

array([[-0.383, -0.095, -0.009, ..., -0.012,  0.   , -0.002],
       [-0.095, -0.38 , -0.102, ..., -0.052,  0.   , -0.024],
       [-0.009, -0.102, -0.412, ...,  0.055, -0.   ,  0.027],
       ...,
       [-0.012, -0.052,  0.055, ..., -0.216, -0.   , -0.043],
       [ 0.   ,  0.   , -0.   , ..., -0.   , -0.216, -0.   ],
       [-0.002, -0.024,  0.027, ..., -0.043, -0.   , -0.157]])

In [31]:
dV=DeltaV(Benz,[[0],[1]])

In [32]:
ap.build_all()
ap.APDFT3([1,0,0,0,0,0])

-242.9419639100907

In [33]:
ap.APDFT3([0,0,1,0,0,0])

-242.94196390984146

In [54]:
(rotate_matrix(dV,Benz,2,ref_site=0)-ap.dV(2))[2:5]

array([[ 0.   ,  0.   ,  0.   , -0.066,  0.   ,  0.118,  0.   ,  0.   ,
         0.   ,  0.306, -0.   ,  0.026, -0.001, -0.021,  0.072,  0.042,
        -0.   , -0.002, -0.   , -0.013,  0.023, -0.012, -0.   , -0.001,
        -0.001, -0.026,  0.002, -0.039, -0.   , -0.003, -0.013, -0.095,
        -0.111, -0.079,  0.   , -0.017],
       [-0.011, -0.118, -0.066,  0.   ,  0.   ,  0.037, -0.186, -0.306,
         0.306,  0.   , -0.   , -0.082, -0.002, -0.053,  0.042, -0.072,
         0.   , -0.007, -0.   , -0.01 , -0.012, -0.023,  0.   , -0.001,
        -0.   , -0.003, -0.039, -0.002,  0.   ,  0.   ,  0.007,  0.034,
        -0.079,  0.111,  0.   ,  0.021],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,  0.   ,
        -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   ,
         0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   , -0.   ,  0.   ]])

In [62]:
p_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "px" in elem]

In [236]:
ang=np.pi/3*2
rm_p=np.asarray([[np.cos(ang),-np.sin(ang)],\
                 [np.sin(ang),np.cos(ang)]])

In [237]:
rm=np.eye(Benz.nao)
for i in p_idxs:
    rm[i:i+2,i:i+2]=rm_p

In [238]:
np.allclose(ap.dV(0),rotate_matrix(rm@ap.dV(4)@rm.T,Benz,0,ref_site=4),atol=1e-3)

True

In [239]:
dm0=BenzHF.make_rdm1()

In [240]:
np.allclose(dm0,rotate_matrix(rm@dm0@rm.T,Benz,0,ref_site=4),atol=1e-3)

True

In [176]:
np.set_printoptions(precision=4)

In [131]:
pt=Benz.aoslice_by_atom()[0,-2]
rpt=Benz.aoslice_by_atom()[2,-2]

In [ ]:
BenzHF.make_rdm1()[pt:pt+5,pt:pt+5],BenzHF.make_rdm1()[rpt:rpt+5,rpt:rpt+5]

In [ ]:
np.cos(np.pi/3)*0.0191,np.sin(np.pi/3)*0.0191

In [ ]:
np.array((0,0,1))

In [ ]:
coords=Benz.atom_coords()

In [ ]:
#symm.Dmatrix.get_euler_angles(coords[0],coords[2])

In [ ]:
#symm.geom.detect_symm(Benz.atom_and_coords())

In [ ]:
AP.mo1(2).shape

In [ ]:
bsym=benz_Symm(Benz)

In [ ]:
bsym.eqs

In [ ]:
coords=Benz.atom_coords()

In [ ]:
coords[::]

In [ ]:
rotate_grad(bsym.eqs[4]['op'].apply(coords),4,ref_site=0)

In [ ]:
rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6)

In [ ]:
AP.af(6)

In [ ]:
np.allclose(rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6),AP.af(6))